# Deep Learning: Mastering Neural Networks - Module 6 Assignment: Deep Convolutional Generative Adverserial Networks (DCGAN)

In this assignment, we will take a look at an implementation of Deep Convolutional Generative Adverserial Networks (DCGAN). GANs were first introduced in the following paper: and have been a very exciting implementation of deep neural networks. Pytorch provides example code for implementing DCGANs and a tutorial is provided here. For this assignment we have adapted the code in the tutorial.

Your task in this assignment will be to train Discriminator and Generator networks to create new Flowers based on the Oxford102Flowers dataset that is available in PyTorch. Since we have not shown an example of training a GAN before, skeleton code will be provided!

Note: training these GANs for 100 Epochs takes ~15min on a GPU enabled Colab notebook.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision.datasets
import torchvision.utils as vutils
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.nn.modules.flatten import Flatten
import time, copy
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

# device config (train our model on GPU if it is available which is much faster)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# These transforms will be performed on every datapoint - in this example we want to transform every
# datapoint to a Tensor datatype, and perform normalization
image_size = 64
transform = transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

flowers_train = torchvision.datasets.Flowers102('', split = "train", transform =transform, download=True)

In [ ]:
# We will create DataLoaders with a batch size of 102, allowing us to use 10 batches per Epoch
batch_size = 102
dataloaders = {'train': DataLoader(flowers_train, batch_size=batch_size)}

dataset_sizes = {'train': len(flowers_train)}
print(f'dataset_sizes = {dataset_sizes}')

## Dataset Visualization

Here we will visualize a grid of sample of our datasets. You can see that there are multiple pictures of each type of flower.

In [ ]:
# Visualization function borrowed from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

# Plot some training images
real_batch = next(iter(dataloaders["train"]))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

## Creating the Models

The Generator and Discriminator networks are alot more complex than a standard CNN. Below we have defined some network parameters that are given in the original paper and the PyTorch tutorial.

In [ ]:
# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

In [ ]:
# TODO: Add the activation functions into the models!
# You should add a LeakyRelu that operates in place with a negative slope of 0.2
# for every activation function in the discriminator EXCEPT the last one, which
# should be a Sigmoid
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.pipeline = nn.Sequential(
            # input is (nc) x 64 x 64

            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            # TODO: Activation function
            <your_code_here>
        )

    def forward(self, input):
        return self.pipeline(input)

# For the generator, you should use regular ReLU functions that operate in place
# for all of the activation functions except for the last one, which should be
# a TanH
class Generator(nn.Module):
    def __init__(self, nc, nz, ngf):
        super(Generator, self).__init__()
        self.pipeline = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            # TODO: Activation function
            <your_code_here>,
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            # TODO: Activation function
            <your_code_here>
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.pipeline(input)

In [ ]:
# custom weights initialization called on netG and netD
# This function initializes the weights of certain layers according to the distributions described in the original paper
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# TODO: Create the Discriminator and Generator (and send to device)
netD =
netG =

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)
netG.apply(weights_init)

# Print the model
print(netD)
print(netG)

## Training the Models

This is training loop is much different than ones we have used in the past and is specialized to train GANs. We will train on our entire training set and leave no images for validation or testing as we are interested in showing our model as much data as possible to improve the Generative capabilities.

In [ ]:
# Training adapted from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

def train_GANS(netD, netG, dataloaders, dataset_sizes, criterion, optimizerD, optimizerG, num_epochs=25):
    since = time.time()

    # Keep track of how loss and accuracy evolves during training
    training_curves = {}
    training_curves['G'] = []
    training_curves['D'] = []
    phase = 'train'

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for _, inputs in enumerate(dataloaders[phase], 0):

            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            optimizerD.zero_grad()
            # Format batch
            real_cpu = inputs[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            label.fill_(fake_label)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch, accumulated (summed) with previous gradients
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Compute error of D as sum over the fake and the real batches
            errD = errD_real + errD_fake
            # Update D
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake).view(-1)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizerG.step()

            training_curves['D'].append(errD.item())
            training_curves['G'].append(errG.item())

            print(f'D Loss: {errD.item():.4f}  G Loss: {errG.item():.4f}')

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return netD, netG, training_curves


In [ ]:
# Number of training epochs
num_epochs = 2 #Try 100 or so, once working

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# TODO: Initialize BCELoss function
criterion =

# Setup Adam optimizers for both G and D
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Train the model. We also will store the results of training to visualize
netD, netG, training_curves = train_GANS(netD, netG, dataloaders, dataset_sizes,
                                     criterion, optimizerD, optimizerG, num_epochs=num_epochs)

## Visualizing Results

Here we can visualize the training curves of our two networks and see some example images that our Generative Model has created!

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(training_curves['G'] ,label="G")
plt.plot(training_curves['D'] ,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloaders['train']))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Generate Fake images
img_list = []
with torch.no_grad():
    fake = netG(fixed_noise).detach().cpu()
    img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

After training for 100 epochs our model is able to generate some images of fake flowers that look somewhat convincing! Try training on a new dataset if you would like to generate something other than flowers!